In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc

---------

In [4]:
from nn_rag import Knowledge
from nn_rag.components.commons import Commons

In [5]:
# create instance of capability
kn = Knowledge.from_memory()

In [8]:
text = ('You took too long. You are not easy to deal with.\n\nPayment Failure/Incorrect Payment.\n\nYou provided '
        'me with incorrect information. Unhappy with delay.\n\nUnsuitable advice. You never answered my question.\n\n'
        'You did not understand my needs.\n\nI have been mis-sold. My details are not accurate.')
arr = pa.array([text], pa.string())
tbl = pa.table([arr], names=['text'])
p = kn.tools.text_to_paragraphs(tbl, has_stats=False)
kn.table_report(p, headers='text', drop=True)

,paragraph_word_count,paragraph_words,index,paragraph_score,paragraph_token_count,paragraph_char_count,paragraph_sentence_count
0,11,[],0,0,12,49,2
1,3,[],1,0,8,34,1
2,9,[],2,0,16,63,2
3,7,[],3,0,12,50,2
4,6,[],4,0,8,32,1
5,9,[],5,0,12,50,2


In [49]:
indices = [2,3]

df = p.to_pandas()
for idx in range(1, df.shape[0]):
    if idx-1 in indices:
        df.loc[idx,'index'] = df.loc[idx-1,'index']
df

,index,paragraph_char_count,paragraph_word_count,paragraph_sentence_count,paragraph_token_count,paragraph_score,paragraph_words,text
0,0,49,11,2,12,0,[],You took too long. You are not easy to deal with.
1,1,34,3,1,8,0,[],Payment Failure/Incorrect Payment.
2,2,63,9,2,16,0,[],You provided me with incorrect information. Unhappy with delay.
3,2,50,7,2,12,0,[],Unsuitable advice. You never answered my question.
4,2,32,6,1,8,0,[],You did not understand my needs.
5,5,50,9,2,12,0,[],I have been mis-sold. My details are not accurate.


AttributeError: 'dict' object has no attribute 'index'

In [35]:
# df = pd.DataFrame(pl)
# df.groupby('index').sum()

In [36]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]